In [91]:
import json
import pandas as pd
import datetime

In [92]:
from pyxirr import xirr

pd.options.display.float_format = '{:,.2f}'.format

In [20]:
#Concatenation des dataframes
dfConcat = pd.read_csv('Data/Solde_Patrimoine.csv')

dfConcat['Date_last_sync'] = pd.to_datetime(dfConcat['Date_last_sync']).dt.tz_localize(None)
dfConcat['Date_extract'] = pd.to_datetime(dfConcat['Date_extract']).dt.tz_localize(None)

In [22]:
dfConcat.dtypes

Date_last_sync    datetime64[ns]
Date_extract      datetime64[ns]
Agregat1_ID               object
Organisme                 object
Produit                   object
Solde                    float64
Week_number                int64
dtype: object

In [21]:
dfConcat

,Date_last_sync,Date_extract,Agregat1_ID,Organisme,Produit,Solde,Week_number
0,2023-04-21 10:34:43.677,2023-04-21 14:01:55.368,1.1,Allianz,Assurance Vie,33391.480230,16
1,2023-04-21 11:57:24.409,2023-04-21 14:01:55.368,2.4,Amundi,PEE / PERCO,20933.062490,16
2,2023-04-21 00:32:38.274,2023-04-21 14:01:55.368,7.1.3,Linxea,Assurance Vie Avenir,19101.573657,16
3,2023-04-21 07:36:05.571,2023-04-21 14:01:55.368,7.1.2,Linxea,Assurance Vie Spirit 2,2158.682589,16
4,2023-04-18 03:45:03.035,2023-04-21 14:01:55.368,5.4,Groupama,PEE / PERCO,3780.696666,16
...,...,...,...,...,...,...,...
139,2023-06-01 13:43:02.443,2023-06-02 01:00:10.504,5.4,Groupama,PEE / PERCO,6536.463488,22
140,2023-06-01 12:25:18.506,2023-06-02 01:00:10.504,6.1,HSBC,Assurance Vie,19470.794018,22
141,2023-06-01 21:08:16.261,2023-06-02 01:00:10.504,3.2,BourseDirect,PEA,154951.772600,22
142,2023-06-01 19:20:36.562,2023-06-02 01:00:10.504,4.3,Boursorama,PEA-PMA,4289.703600,22


In [4]:
dfConcat['Date_extract'].dt.isocalendar().week

0     16
1     16
2     16
3     16
4     16
      ..
76    21
77    21
78    21
79    21
80    21
Name: week, Length: 81, dtype: UInt32

In [5]:
dfComptes = pd.read_excel('Param/Description_Comptes.xlsx')

In [8]:
# Ouvrir le fichier UC en mode lecture
with open("Data/BalanceHeritageUC.json", "r") as f:
    # Charger le contenu du fichier en tant qu'objet JSON
    json_data_UC = json.load(f)

BalancePatrimoine = {'ID':[],"Date_last_sync" :[], "Date_extract" : [],"Nom":[], "Type":[] ,"Solde" : []}

# Afficher l'objet JSON UC sous forme de dictionnaire Python
for i in range(len(json_data_UC["result"])):
    
    BalancePatrimoine["ID"].append(json_data_UC["result"][i]["id"])

    BalancePatrimoine["Date_last_sync"].append(json_data_UC["result"][i]["last_sync"])

    BalancePatrimoine["Date_extract"].append(datetime.datetime.now())

    BalancePatrimoine["Nom"].append(json_data_UC["result"][i]["name"])

    BalancePatrimoine["Type"].append("UC")

    BalancePatrimoine["Solde"].append(json_data_UC["result"][i]["balance"])
    

# Convertir le dictionnaire en DataFrame
df = pd.DataFrame(BalancePatrimoine)

In [9]:
dfAgregat = pd.merge(df, dfComptes, on='ID', how='left')

In [14]:
dfGrouped = dfAgregat.groupby('Agregat1').agg(Date_last_sync = ('Date_last_sync', 'first'), Date_extract = ('Date_extract', 'first'), Agregat1_ID = ('Agregat1_ID', 'first') ,Organisme =('Organisme', 'first'), Produit =('Produit', 'first'), Solde = ('Solde', 'sum'))


In [16]:
dfGrouped['Week_number'] = dfGrouped['Date_extract'].dt.isocalendar().week

In [17]:
dfGrouped

,Date_last_sync,Date_extract,Agregat1_ID,Organisme,Produit,Solde,Week_number
Agregat1,,,,,,,
Allianz,2023-05-28T11:59:32.669Z,2023-05-30 22:31:05.774571,1.1,Allianz,Assurance Vie,33426.019091,22
Allianz PEE / PERCO,2023-05-28T16:08:52.971Z,2023-05-30 22:31:05.774558,2.4,Amundi,PEE / PERCO,20524.976557,22
Assurance Vie Avenir,2023-05-28T13:45:57.140Z,2023-05-30 22:31:05.774587,7.1.3,Linxea,Assurance Vie Avenir,19011.735471,22
Assurance Vie Spirit 2,2023-05-28T16:16:39.076Z,2023-05-30 22:31:05.774585,7.1.2,Linxea,Assurance Vie Spirit 2,2164.344155,22
Groupama PEE / PERCO,2023-05-28T15:20:39.037Z,2023-05-30 22:31:05.774574,5.4,Groupama,PEE / PERCO,6464.014000,22
HSBC,2023-05-28T13:18:26.815Z,2023-05-30 22:31:05.774593,6.1,HSBC,Assurance Vie,19407.494889,22
PEA,2023-05-28T10:20:38.764Z,2023-05-30 22:31:05.774567,3.2,BourseDirect,PEA,155519.718000,22
PEA-PMA,2023-05-28T11:10:26.918Z,2023-05-30 22:31:05.774581,4.3,Boursorama,PEA-PMA,4239.826800,22
Yomoni,2023-05-28T11:21:27.283Z,2023-05-30 22:31:05.774583,8.1,Yomoni,Assurance Vie,9375.492241,22


## Rajout dans l'historique numéro semaine et ID aggregat1

In [16]:
dfHisto = pd.read_excel('Data/Solde_Patrimoine.xlsx')

dfHisto['Date_last_sync'] = pd.to_datetime(dfHisto['Date_last_sync']).dt.tz_localize(None)
dfHisto['Date_extract'] = pd.to_datetime(dfHisto['Date_extract']).dt.tz_localize(None)

In [18]:
dfHisto.dtypes

Date_last_sync    datetime64[ns]
Date_extract      datetime64[ns]
Agregat1_ID               object
Organisme                 object
Produit                   object
Solde                    float64
Week_number                int64
dtype: object

In [19]:
dfHisto.to_csv('Data/Solde_Patrimoine.csv', sep=',', encoding='utf-8', index=False)

# Calcul du rendement annuel YDT

In [6]:
dfXirrYdt = pd.read_excel('Data/Test XIRR_Patrimoine.xlsx')

In [104]:
TRI_Patrimoine = {'Agregat1_ID':[], "Date" : [],"Organisme":[], "Produit":[] ,"Rentabilité YTD" : []}

In [105]:
for i in dfXirrYdt[['Agregat1_ID']].drop_duplicates(keep = 'first')['Agregat1_ID']:
    TRI_Patrimoine['Agregat1_ID'].append(i)
    
    TRI_Patrimoine['Date'].append("01/06/2023")
    
    for j in dfXirrYdt[dfXirrYdt['Agregat1_ID'] == i][['Organisme']].drop_duplicates(keep = 'first')['Organisme']:
        TRI_Patrimoine['Organisme'].append(j)
 
    for k in dfXirrYdt[dfXirrYdt['Agregat1_ID'] == i][['Produit']].drop_duplicates(keep = 'first')['Produit']:
        TRI_Patrimoine['Produit'].append(k)
    
    TRI_Patrimoine['Rentabilité YTD'].append(xirr(dfXirrYdt[dfXirrYdt['Agregat1_ID'] == i][['Date','Solde']]))
    


In [107]:
dfRentabilite = pd.DataFrame(TRI_Patrimoine)

In [108]:
dfRentabilite

,Agregat1_ID,Date,Organisme,Produit,Rentabilité YTD
0,1.1,01/06/2023,Allianz,Assurance Vie,0.091410
1,2.4,01/06/2023,Amundi,PEE / PERCO,0.414457
2,7.1.3,01/06/2023,Linxea,Assurance Vie Avenir,0.074793
3,7.1.2,01/06/2023,Linxea,Assurance Vie Spirit 2,0.032162
4,5.4,01/06/2023,Groupama,PEE / PERCO,0.191304
5,6.1,01/06/2023,HSBC,Assurance Vie,0.081968
6,3.2,01/06/2023,BourseDirect,PEA,0.071748
7,4.3,01/06/2023,Boursorama,PEA-PMA,0.113558
8,8.1,01/06/2023,Yomoni,Assurance Vie,0.155508


In [40]:
# Bourse Direct PEA
"%.1f%%" % (xirr(dfXirrYdt[dfXirrYdt['Agregat1_ID'] == "3.2"][['Date','Solde']])*100)

'7.2%'

# Création du dataframe pour calcul de la rentabilité

In [149]:
dfParam = pd.read_excel('Param/Description_Comptes.xlsx')

In [122]:
dfParam[['Agregat1_ID']].drop_duplicates(keep = 'first')[['Agregat1_ID']]

,Agregat1_ID
0,1.1
1,2.4
4,3.2
5,4.3
6,5.4
8,6.1
9,7.1.2
10,7.1.3
11,8.1


## Production fichier solde YDT

In [152]:
dfSolde = pd.read_csv('Data/Solde_Patrimoine.csv')

dfSolde['Date_last_sync'] = pd.to_datetime(dfSolde['Date_last_sync']).dt.tz_localize(None)
dfSolde['Date_extract'] = pd.to_datetime(dfSolde['Date_extract']).dt.tz_localize(None)


In [153]:
dfSolde

,Date_last_sync,Date_extract,Agregat1_ID,Organisme,Produit,Solde,Week_number
0,2023-04-21 10:34:43.677,2023-04-21 14:01:55.368000,1.1,Allianz,Assurance Vie,33391.480230,16
1,2023-04-21 11:57:24.409,2023-04-21 14:01:55.368000,2.4,Amundi,PEE / PERCO,20933.062490,16
2,2023-04-21 00:32:38.274,2023-04-21 14:01:55.368000,7.1.3,Linxea,Assurance Vie Avenir,19101.573657,16
3,2023-04-21 07:36:05.571,2023-04-21 14:01:55.368000,7.1.2,Linxea,Assurance Vie Spirit 2,2158.682589,16
4,2023-04-18 03:45:03.035,2023-04-21 14:01:55.368000,5.4,Groupama,PEE / PERCO,3780.696666,16
...,...,...,...,...,...,...,...
193,2023-06-02 06:58:01.009,2023-06-02 15:04:25.299382,5.4,Groupama,PEE / PERCO,6531.366469,22
194,2023-06-02 02:28:38.462,2023-06-02 15:04:25.299386,6.1,HSBC,Assurance Vie,19420.815485,22
195,2023-06-01 21:08:16.261,2023-06-02 15:04:25.299377,3.2,BourseDirect,PEA,155773.942783,22
196,2023-06-02 04:52:00.066,2023-06-02 15:04:25.299380,4.3,Boursorama,PEA-PMA,4289.914800,22


In [154]:
# get all differences with date as values

TimeYDT = datetime.datetime(1900, 1, 1)

for i in dfSolde['Date_extract']:
        if (i - datetime.datetime.now()) > (TimeYDT - datetime.datetime.now()):
            TimeYDT = i


In [155]:

dfYDTSolde = dfSolde[pd.to_datetime(dfSolde['Date_extract']).dt.date == TimeYDT.date()].drop_duplicates(subset='Agregat1_ID', keep="first")


In [156]:
dfYDTSolde["Type"] = "Solde"

In [157]:
dfYDTSolde.rename(columns = {'Date_extract':'Date'}, inplace = True)

In [158]:
dfYDTSolde = dfYDTSolde.drop(columns=['Date_last_sync'])

## Production fichier solde ouverture

In [159]:
TimeOuverture = datetime.date(2023, 1, 1)

In [160]:
dfOuverture = pd.read_excel('Data/Ouverture_Patrimoine.xlsx')

In [161]:

dfYDTOuverture = dfOuverture[pd.to_datetime(dfOuverture['Date']).dt.date == TimeOuverture].drop_duplicates(subset='Agregat1_ID', keep="first")


In [162]:
dfYDTOuverture['Solde'] = dfYDTOuverture['Solde'] * -1

In [163]:
dfYDTOuverture['Type'] = "Ouverture"

## Production fichier mouvement

In [164]:
dfMouvement = pd.read_excel('Data/Mouvement_Patrimoine.xlsx')

In [165]:

dfMouvementYDT = dfMouvement[(pd.to_datetime(dfMouvement['Date']).dt.date >= TimeOuverture) & (pd.to_datetime(dfMouvement['Date']).dt.date <= TimeYDT.date())]


In [166]:

dfMouvementYDT["Solde"] = dfMouvementYDT["Solde"] * -1

In [167]:
dfMouvementYDT["Type"] = "Mouvement"

In [168]:
# Passer le type Agregat1_ID en objet ou integer

dfMouvementYDT["Agregat1_ID"] = dfMouvementYDT["Agregat1_ID"].astype(str)

## Concat des 3 dataframes

In [169]:
dfXirrYdt = pd.concat([dfYDTOuverture, dfMouvementYDT, dfYDTSolde])

# Création du dataframe pour calcul de la rentabilité

In [240]:
TRI_Patrimoine = {'Agregat1_ID':[], "Date" : [],"Organisme":[], "Produit":[] ,"TRI YTD" : [],"Rendement" :[], "Solde":[], "Total Mouvements":[]}


In [241]:

for i in dfXirrYdt[['Agregat1_ID']].drop_duplicates(keep = 'first')['Agregat1_ID']:
    TRI_Patrimoine['Agregat1_ID'].append(i)
    
    TRI_Patrimoine['Date'].append(TimeYDT)
    
    for j in dfXirrYdt[dfXirrYdt['Agregat1_ID'] == i][['Organisme']].drop_duplicates(keep = 'first')['Organisme']:
        TRI_Patrimoine['Organisme'].append(j)
 
    for k in dfXirrYdt[dfXirrYdt['Agregat1_ID'] == i][['Produit']].drop_duplicates(keep = 'first')['Produit']:
        TRI_Patrimoine['Produit'].append(k)
    
    TRI_Patrimoine['TRI YTD'].append(xirr(dfXirrYdt[dfXirrYdt['Agregat1_ID'] == i][['Date','Solde']]))
    
    TRI_Patrimoine['Rendement'].append(dfXirrYdt[dfXirrYdt['Agregat1_ID'] == i]['Solde'].sum())
    
    TRI_Patrimoine['Solde'].append(dfXirrYdt[(dfXirrYdt['Agregat1_ID'] == i) & (dfXirrYdt['Type'] == "Solde")]["Solde"].sum())
    
    TRI_Patrimoine['Total Mouvements'].append(dfXirrYdt[(dfXirrYdt['Agregat1_ID'] == i) & (dfXirrYdt['Type'] == "Mouvement")]["Solde"].sum() * -1)
    

In [244]:
TRI_Patrimoine['Agregat1_ID'].append("0.0")
    
TRI_Patrimoine['Date'].append(TimeYDT)

TRI_Patrimoine['Organisme'].append("Total")

TRI_Patrimoine['Produit'].append(" - ")

TRI_Patrimoine['TRI YTD'].append(xirr(dfXirrYdt[['Date','Solde']]))

TRI_Patrimoine['Rendement'].append(dfXirrYdt['Solde'].sum())

TRI_Patrimoine['Solde'].append(dfXirrYdt[(dfXirrYdt['Type'] == "Solde")]["Solde"].sum())

TRI_Patrimoine['Total Mouvements'].append(dfXirrYdt[(dfXirrYdt['Type'] == "Mouvement")]["Solde"].sum() * -1)


In [245]:
dfResultatXirrYDT = pd.DataFrame(TRI_Patrimoine)

In [246]:
dfResultatXirrYDT

,Agregat1_ID,Date,Organisme,Produit,TRI YTD,Rendement,Solde,Total Mouvements
0,1.1,2023-06-02 15:04:25.299386,Allianz,Assurance Vie,0.09,"1,189.44","33,458.44",515.00
1,2.4,2023-06-02 15:04:25.299386,Amundi,PEE / PERCO,0.06,514.14,"22,417.14","2,500.00"
2,7.1.3,2023-06-02 15:04:25.299386,Linxea,Assurance Vie Avenir,0.07,560.76,"18,950.76",-0.00
3,7.1.2,2023-06-02 15:04:25.299386,Linxea,Assurance Vie Spirit 2,0.03,29.36,"2,165.36",-0.00
4,5.4,2023-06-02 15:04:25.299386,Groupama,PEE / PERCO,0.19,295.46,"6,536.46","2,636.00"
5,6.1,2023-06-02 15:04:25.299386,HSBC,Assurance Vie,0.09,676.79,"19,470.79",-0.00
6,3.2,2023-06-02 15:04:25.299386,BourseDirect,PEA,0.06,"3,471.77","154,951.77","18,700.00"
7,4.3,2023-06-02 15:04:25.299386,Boursorama,PEA-PMA,0.11,187.70,"4,289.70",-0.00
8,8.1,2023-06-02 15:04:25.299386,Yomoni,Assurance Vie,0.18,469.20,"9,397.20","3,700.00"
9,0.0,2023-06-02 15:04:25.299386,Total,-,0.07,"7,394.64","271,637.64","28,051.00"


In [247]:
dfResultatXirrYDT.to_excel('Data/TRI_Ydt.xlsx', sheet_name='TRI YDT', float_format = "%0.3f", index=False)